In [ ]:
import gc
gc.collect()

In [ ]:
# Import comet_ml at the top of your file
# from comet_ml import Experiment


from matplotlib import pyplot as plt
# from matplotlib import style
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import swifter
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import itertools
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras as k
import tensorflow as tf
# import torch
from scipy import stats
import nltk



# Create an experiment with your api key
# experiment = Experiment(
#     api_key="BKac2uRt0FMAlheXf6HClaZhD",
#     project_name="general",
#     workspace="sarthak7034",
# )


In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english') 

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('gpu')))
# torch.cuda.is_available()

In [ ]:
# # Function to plot the confusion matrix (code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)


# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
    
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [ ]:
# experiment.end()

In [ ]:
#Read dataset
data = pd.read_csv("./all_data.csv").drop("Unnamed: 0", axis=1)

In [ ]:
gg = data.loc[:,['title','text','type']]
gg

## Cleaning Text

In [ ]:
def clean_text(text):
    #Remove URLs
    text = re.sub(r"http\S+", "", text)
    #Tokenize
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokens = tokenizer.tokenize(text)
    # remove stopwords
    tokens = [w for w in tokens if not w in stop_words]     # Experiment with removing it!!!
    #Remove non alphanumerica characters
    words = [word for word in tokens if word.isalpha()]    
    return words

scaler = MinMaxScaler()

In [ ]:
#Applying lamdba function to clean text
data['text'] = data.swifter.apply(lambda row: clean_text(row['text']), axis=1)
#Clean title
data['title'] = data.swifter.apply(lambda row: clean_text(row['title']), axis=1)


In [ ]:
#Replacing row with only text
data = data[data['title'].str.len() >= 1]
data = data[data['text'].str.len() >= 1]

In [ ]:
# df1 = data.loc[:,['anger','anticipation','disgust','fear','joy','sadness','surprise','trust','negative','positive']]

# # Zero Score Dataframe Normalization
# MaxMin_df = stats.zscore(df1)
# MaxMin_df.head()

In [ ]:
#Displaying the new dataset which we will be working with
df2 = data.loc[:,['title','text','type']]

In [ ]:
df2

In [ ]:
df2 = df2[df2['text'].map(len) >= 1]
#Reset index
df2 = df2.reset_index().drop("index", axis=1)
df2


In [ ]:
# Whether any values are missing
# df2.isnull().sum()
# df1.isnull().sum()

In [ ]:
# How many fake and real articles?
print(df2.groupby(['type'])['title'].count())
df2.groupby(['type'])['title'].count().plot(kind="bar")
plt.show()

## Word2Vec

In [ ]:
model_embed = KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin.gz",binary=True)

In [ ]:
vocab_length = len(model_embed.key_to_index.keys())
print(vocab_length)                                         #Input dimension size(3000000)

In [ ]:
# Applying it to the columns 

df2['title'] = df2.swifter.apply(lambda r: [model_embed.key_to_index[x] for x in r['title'] if x in model_embed.key_to_index], axis=1)
df2['text'] = df2.swifter.apply(lambda r: [model_embed.key_to_index[x] for x in r['text'] if x in model_embed.key_to_index], axis=1)

In [ ]:
# # One-hot encoding
# encoder = LabelBinarizer().fit(list(df2['type']))
# df2['type'] = df2.swifter.apply(lambda r: encoder.transform([r['type']])[0], axis=1)

variable_name = {'fake' : 0 , 'real' : 1 }
df2['type'] = df2['type'].map(variable_name)

In [ ]:
df2 = df2[df2['text'].map(len) >= 1]
#Reset index
df2 = df2.reset_index().drop("index", axis=1)
df2

In [ ]:
# Shifting all to a new dataframe
df3 = df2.loc[:,['title','text','type']]

In [ ]:
# Check the highest length of text and title:
max_tokens_text = 13000
max_tokens_title = 48

## Padding

In [ ]:
df3['text'] = list(pad_sequences((df2['text']), maxlen=max_tokens_text,padding='post'))
df3['title'] = list(pad_sequences(df2['title'], maxlen=max_tokens_title,padding='post'))

In [ ]:
#Python object serialized
df3.to_pickle('./model_data.pickle')

# Building Model

## Loading Pickle 

In [76]:
data_pickle = pd.read_pickle('./model_data.pickle')
data_pickle = data_pickle.dropna()


In [77]:
#Train - test
X_train, X_test, y_train, y_test = train_test_split(
    data_pickle[['text', 'title']], data_pickle['type'], test_size=0.2, random_state=42)

#Train - valid
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)
 

train_fit = [np.asarray(X_train['title'].tolist()), 
             np.asarray(X_train['text'].tolist())
             ]

valid_fit = [np.asarray(X_valid['title'].tolist()), 
             np.asarray(X_valid['text'].tolist())
             ]

test_fit = [np.asarray(X_test['title'].tolist()), 
             np.asarray(X_test['text'].tolist())
             ]    




## Binary_crossentropy

In [78]:
from numpy.random import seed

seed(50)
tf.random.set_seed(50)
k.backend.clear_session()

In [82]:
#input_title
title_input = k.layers.Input(shape=(max_tokens_title,))
inp1 = k.layers.Embedding(input_dim=3000000,output_dim=300,trainable=False)(title_input)
x1 = k.layers.Dropout(0.2)(inp1)

#Added
x1 = k.layers.Conv1D(filters = 32, kernel_size=4, strides=2, activation='relu')(x1)
x1 = k.layers.MaxPool1D(pool_size = 2)(x1)
x1 = k.layers.Flatten()(x1)
x1 = k.layers.Dense(50, activation='relu', kernel_regularizer='l2')(x1)

#input_Text
text_input = k.layers.Input(shape=(max_tokens_text,))
inp2 = k.layers.Embedding(input_dim=3000000,output_dim=300,trainable=False)(text_input)
x2 = k.layers.Dropout(0.2)(inp2)

x2 = k.layers.Conv1D(filters = 64, kernel_size = 4, strides = 2, activation='relu')(x2)
x2 = k.layers.MaxPool1D(pool_size = 4)(x2)

#Added
# x2 = k.layers.Conv1D(filters = 80, kernel_size = 16, strides = 4, activation='relu')(inp2)
# x2 = k.layers.MaxPool1D(pool_size = 4)(x2)
x2 = k.layers.Flatten()(x2)
x2 = k.layers.Dense(100, activation='relu', kernel_regularizer='l2')(x2)


#Merge
x = k.layers.concatenate([x1, x2])

#Common part
# x = k.layers.Dense(50, activation='relu')(x)
# x = k.layers.Dropout(0.2)(x)
x = k.layers.Dense(50, activation='relu')(x)
x = k.layers.Dropout(0.2)(x)
out = k.layers.Dense(1, activation='sigmoid')(x)


#Build model
model = k.models.Model(inputs=[title_input, text_input], outputs=[out])

model.compile(k.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 48)]         0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 13000)]      0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 48, 300)      900000000   ['input_7[0][0]']                
                                                                                                  
 embedding_7 (Embedding)        (None, 13000, 300)   900000000   ['input_8[0][0]']                
                                                                                            

## Fitting the Model

In [83]:
history = model.fit(x=train_fit, y=np.asarray(y_train.tolist()), batch_size=128, epochs=20,
        callbacks = [k.callbacks.EarlyStopping(monitor='val_loss', patience=4)], 
        validation_data=(valid_fit, np.array(y_valid.tolist())))

Epoch 1/20
99/99 [==============================] - 306s 3s/step - loss: 0.9944 - acc: 0.6905 - val_loss: 0.5832 - val_acc: 0.7697
Epoch 2/20
99/99 [==============================] - 309s 3s/step - loss: 0.5125 - acc: 0.8144 - val_loss: 0.4444 - val_acc: 0.8574
Epoch 3/20
99/99 [==============================] - 309s 3s/step - loss: 0.4329 - acc: 0.8575 - val_loss: 0.4179 - val_acc: 0.8605
Epoch 4/20
99/99 [==============================] - 308s 3s/step - loss: 0.3714 - acc: 0.8887 - val_loss: 0.3421 - val_acc: 0.8999
Epoch 5/20
99/99 [==============================] - 299s 3s/step - loss: 0.3187 - acc: 0.9142 - val_loss: 0.3424 - val_acc: 0.9003
Epoch 6/20
99/99 [==============================] - 289s 3s/step - loss: 0.2831 - acc: 0.9267 - val_loss: 0.3064 - val_acc: 0.9085
Epoch 7/20
99/99 [==============================] - 313s 3s/step - loss: 0.2595 - acc: 0.9364 - val_loss: 0.2881 - val_acc: 0.9196
Epoch 8/20
99/99 [==============================] - 298s 3s/step - loss: 0.2291 - a

In [84]:
test_pred = model.predict(test_fit)

In [85]:
report = classification_report(np.array(y_test.tolist()), test_pred.round())
print(report)

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2354
           1       0.96      0.89      0.92      1581

    accuracy                           0.94      3935
   macro avg       0.94      0.93      0.94      3935
weighted avg       0.94      0.94      0.94      3935



## Categorical_crossentropy

In [ ]:
from numpy.random import seed

seed(50)
tf.random.set_seed(50)
k.backend.clear_session()

In [ ]:
#input_title
title_input = k.layers.Input(shape=(max_tokens_title,))
inp1 = k.layers.Embedding(input_dim=3000000,output_dim=300,trainable=False)(title_input)

x = k.layers.Conv1D(filters = 5, kernel_size=4, strides=2, activation='relu')(inp1)
x = k.layers.MaxPool1D(pool_size = 2)(x)
x = k.layers.Flatten()(x)
x = k.layers.Dense(50, activation='relu',  kernel_regularizer='l2')(x)

#input_content
text_input = k.layers.Input(shape=(max_tokens_text,))
inp2 = k.layers.Embedding(input_dim=3000000,output_dim=300,trainable=False)(text_input)

x2 = k.layers.Conv1D(filters = 40, kernel_size = 16, strides = 2, activation='relu')(inp2)
x2 = k.layers.MaxPool1D(pool_size = 4 )(x2)

#Added
x2 = k.layers.Conv1D(filters = 80, kernel_size = 16, strides = 4, activation='relu')(inp2)
x2 = k.layers.MaxPool1D(pool_size = 4, )(x2)
x2 = k.layers.Flatten()(x2)
x2 = k.layers.Dense(100, activation='relu', kernel_regularizer='l2')(x2)


#Merge
x = k.layers.concatenate([x, x2])

#Common part
x = k.layers.Dense(50, activation='relu')(x)
x = k.layers.Dropout(0.2)(x)
x = k.layers.Dense(50, activation='relu')(x)
x = k.layers.Dropout(0.2)(x)
out = k.layers.Dense(4, activation='softmax')(x)


#Build model
model = k.models.Model(inputs=[title_input, text_input], outputs=[out])

model.compile(k.optimizers.RMSprop(), loss='sparse_categorical_crossentropy', metrics=['acc'])

model.summary()

## Fitting the Model

In [ ]:
history = model.fit(x=train_fit, y=np.asarray(y_train.tolist()), batch_size=128, epochs=30,
        callbacks = [k.callbacks.EarlyStopping(monitor='val_loss', patience=4)], 
        validation_data=(valid_fit, np.array(y_valid.tolist())))

In [ ]:
test_pred = model.predict(test_fit)

In [ ]:
# get predicted indices (ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets)
test_pred = np.argmax(test_pred, 1)

In [ ]:
report = classification_report(np.array(y_test.tolist()), test_pred.round())
print(report)